In [ ]:
@file:DependsOn("/data/repos/actin-personalization/ncr/target/ncr-local-SNAPSHOT-jar-with-dependencies.jar")

import com.hartwig.actin.personalization.datamodel.PatientRecord
import com.hartwig.actin.personalization.ncr.datamodel.NcrRecord
import com.hartwig.actin.personalization.ncr.interpretation.PatientRecordFactory
import com.hartwig.actin.personalization.ncr.serialization.NcrDataReader

val records = NcrDataReader.read("/data/patient_like_me/ncr/K23244.csv")

In [ ]:
// 1 DIA record per patient-tumor pair:
println(records.groupBy { Pair(it.identification.keyNkr, it.identification.keyZid) }
    .values.map { epiList -> epiList.count { it.identification.epis == "DIA" } }
    .groupBy { it }
    .mapValues { (_, values) -> values.size })

In [ ]:
val patients = PatientRecordFactory.create(records)

In [ ]:
patients.size

In [ ]:
import com.hartwig.actin.personalization.datamodel.Episode
import com.hartwig.actin.personalization.datamodel.Drug
import com.hartwig.actin.personalization.datamodel.SystemicTreatmentComponent
import com.hartwig.actin.personalization.datamodel.SystemicTreatmentScheme

fun treatmentSchemesForTumorEpisodes(episodes: List<Episode>) =
    episodes.flatMap { e -> e.systemicTreatmentSchemes!! }

fun schemeToTreatmentNameSet(scheme: SystemicTreatmentScheme) = 
    scheme.treatmentComponents.map(SystemicTreatmentComponent::drug)
        .map { 
            if (it == Drug.TEGAFUR || it == Drug.TEGAFUR_OR_GIMERACIL_OR_OTERACIL) {
                Drug.FLUOROURACIL
            } else it
        }
        .toSet()

In [ ]:
// Unique treatment schemes by treatment name sets

val treatmentSchemeSets = patients.flatMap { it.episodesPerTumorOfInterest.values }
    .flatMap(::treatmentSchemesForTumorEpisodes)
    .map(::schemeToTreatmentNameSet)
    .groupBy({ it })
    .mapValues { (_, values) -> values.count() }
    .entries.sortedByDescending { (_, count) -> count }
    .takeWhile { (_, count) -> count >= 10 }
    
println(treatmentSchemeSets.map { (names, count) -> "$names: $count" }.joinToString("\n"))
println(treatmentSchemeSets.size)


In [ ]:
// Unique patient tumors by treatment name sets in all schemes

val tumorTreatmentSets = patients.flatMap { it.episodesPerTumorOfInterest.values }
    .map { treatmentSchemesForTumorEpisodes(it).map(::schemeToTreatmentNameSet).distinct() }
    .groupBy({ it })
    .mapValues { (_, values) -> values.count() }
    .entries.sortedByDescending { (_, count) -> count }
    //.takeWhile { (_, count) -> count >= 10 }

In [ ]:
println(tumorTreatmentSets.map { (names, count) -> "$names: $count" }.joinToString("\n"))
println(tumorTreatmentSets.size)